In [10]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Load data
pc3 = pd.read_csv(r'/content/pc3_csv.csv')
print('pc3 shape: ', pc3.shape)
pc3.head()

pc3 shape:  (1563, 38)


,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,LABEL
0,2,1,0,0,0,0,1,0.10,0,0.00,...,2,0.08,28,29,9,5,13,0.00,10,False
1,1,1,4,0,0,0,1,0.07,0,0.00,...,6,0.06,52,55,26,14,16,0.00,14,False
2,27,19,1,4,13,26,11,0.26,12,2.17,...,25,0.13,58,78,30,24,83,30.91,42,False
3,2,17,2,0,0,24,9,0.47,8,3.00,...,25,0.41,73,81,23,20,22,0.00,19,False
4,6,1,1,0,2,0,1,0.11,0,0.00,...,3,0.06,19,23,15,7,18,18.18,9,False


In [12]:
pc3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   LOC_BLANK                        1563 non-null   int64  
 1   BRANCH_COUNT                     1563 non-null   int64  
 2   CALL_PAIRS                       1563 non-null   int64  
 3   LOC_CODE_AND_COMMENT             1563 non-null   int64  
 4   LOC_COMMENTS                     1563 non-null   int64  
 5   CONDITION_COUNT                  1563 non-null   int64  
 6   CYCLOMATIC_COMPLEXITY            1563 non-null   int64  
 7   CYCLOMATIC_DENSITY               1563 non-null   float64
 8   DECISION_COUNT                   1563 non-null   int64  
 9   DECISION_DENSITY                 1563 non-null   float64
 10  DESIGN_COMPLEXITY                1563 non-null   int64  
 11  DESIGN_DENSITY                   1563 non-null   float64
 12  EDGE_COUNT          

In [13]:
target = 'LABEL'
y = pc3[target]
x=pc3.drop(['LABEL'], axis=1)
x.shape

(1563, 37)

after running " *from mlxtend.feature_selection import SequentialFeatureSelector as SFS* " this command you might get error this is because mlxtend is trying to import joblib from sklearn but it no longer exist there so fix this issue go to the file location (there will a file location link in the error message like this " /usr/local/lib/python3.7/dist-packages/mlxtend/feature_selection/sequential_feature_selector.py <module> " click on this location link)
then it will show you code written in that file then you just have to look for this line " from sklearn.externals.joblib import Parallel, delayed " and edit it to " from joblib import Parallel, delayed " and then save it by pressing ctrl+s. then run the code again. Again it will give error but this time the location link will be different and you have to again edit the file in same way as done priviouslly and save it and run the code again and this will solve the error.

In [14]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.svm import SVC
svc = SVC()


In [15]:
efs = EFS(svc, # can use any any learning method 
          min_features=1,# can set any limit from 1 to less then or equal to max_feature value
          max_features=4, # can set any value less then toatal number of features but greater then or equal to min_features
                          # higer the difference between the min and max value higher the time taken to select features
          scoring='accuracy',
          print_progress=True,
          cv=2
          ).fit(np.array(x[x.columns[0:5]].fillna(0)), y)
                                    # set value of columns like [0:x] where x is equal to value of max_features

Features: 30/30

In [16]:
efs.best_idx_

(0,)

In [17]:
x_reduced = efs.transform(x)
x_reduced.shape

(1563, 1)

In [18]:
y.shape

(1563,)

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1563, 37)
(1563,)
-------------------
(1094, 1)
(1094,)
(469, 1)
(469,)


In [20]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.908315565031983
Th confusion matrix of SVM:
  [[426   0]
 [ 43   0]]


In [21]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[426   0]
 [ 43   0]]
              precision    recall  f1-score   support

       False       0.91      1.00      0.95       426
        True       0.00      0.00      0.00        43

    accuracy                           0.91       469
   macro avg       0.45      0.50      0.48       469
weighted avg       0.83      0.91      0.86       469



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8933901918976546
The confusion matrix of KNN:
  [[413  13]
 [ 37   6]]
[[413  13]
 [ 37   6]]
              precision    recall  f1-score   support

       False       0.92      0.97      0.94       426
        True       0.32      0.14      0.19        43

    accuracy                           0.89       469
   macro avg       0.62      0.55      0.57       469
weighted avg       0.86      0.89      0.87       469



In [23]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.9040511727078892
[[418   8]
 [ 37   6]]
              precision    recall  f1-score   support

       False       0.92      0.98      0.95       426
        True       0.43      0.14      0.21        43

    accuracy                           0.90       469
   macro avg       0.67      0.56      0.58       469
weighted avg       0.87      0.90      0.88       469



In [24]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.886654478976234

In [25]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.93      0.97      0.95       426
        True       0.43      0.23      0.30        43

    accuracy                           0.90       469
   macro avg       0.68      0.60      0.63       469
weighted avg       0.88      0.90      0.89       469

[[413  13]
 [ 33  10]]
